<a href="https://colab.research.google.com/github/aekanun2020/2025-AdvancedRAG/blob/main/SENT_Hands_on_2_putSparseVector_into_Opensearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## !!! ก่อนรันโค้ดนี้ ผู้เรียน จำเป็นต้องเปลี่ยน OPENSEARCH_INDEX ให้เป็นชื่อของตนเอง
## ใน format: yourname_doc_index (ตัวพิมพ์เล็ก)!!!

# ติดตั้ง LlamaIndex และ dependencies
!pip install llama-index -q
!pip install llama-index-embeddings-huggingface -q
!pip install llama-index-vector-stores-opensearch -q
!pip install requests -q
!pip install nest_asyncio -q

# Import modules
import os
import torch
import urllib.request
import pickle
import requests
import nest_asyncio
import json
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.opensearch import OpensearchVectorStore, OpensearchVectorClient
from llama_index.core.vector_stores.types import VectorStoreQueryMode

# Apply nest_asyncio to avoid runtime errors
nest_asyncio.apply()

# กำหนดค่าสำหรับ OpenSearch
OPENSEARCH_ENDPOINT = "http://34.41.37.53:9200"
OPENSEARCH_INDEX = "aekanun_doc_index"
TEXT_FIELD = "content"
EMBEDDING_FIELD = "embedding"

# Check if CUDA is available for GPU acceleration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"ใช้อุปกรณ์: {device}")

# ฟังก์ชันสำหรับสร้าง hybrid search pipeline
def create_hybrid_search_pipeline():
    pipeline_url = f"{OPENSEARCH_ENDPOINT}/_search/pipeline/hybrid-search-pipeline"
    headers = {'Content-Type': 'application/json'}

    pipeline_config = {
        "description": "Pipeline for hybrid search",
        "phase_results_processors": [
            {
                "normalization-processor": {
                    "normalization": {
                        "technique": "min_max"
                    },
                    "combination": {
                        "technique": "harmonic_mean",
                        "parameters": {
                            "weights": [0.3, 0.7]
                        }
                    }
                }
            }
        ]
    }

    try:
        response = requests.put(pipeline_url, headers=headers, data=json.dumps(pipeline_config))
        if response.status_code in [200, 201]:
            print(f"สร้าง hybrid search pipeline สำเร็จ: {response.json()}")
        else:
            print(f"ไม่สามารถสร้าง pipeline ได้: {response.text}")
    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการสร้าง pipeline: {e}")

# ฟังก์ชันดาวน์โหลด corpus
def download_corpus():
    os.makedirs('./corpus_input', exist_ok=True)
    urls = [
        ("https://storage.googleapis.com/llm-course/md/1.md", "./corpus_input/1.md"),
        ("https://storage.googleapis.com/llm-course/md/2.md", "./corpus_input/2.md"),
        ("https://storage.googleapis.com/llm-course/md/44.md", "./corpus_input/44.md"),
        ("https://storage.googleapis.com/llm-course/md/5555.md", "./corpus_input/5555.md")
    ]
    for url, path in urls:
        if not os.path.exists(path):
            print(f"กำลังดาวน์โหลด {url} ไปยัง {path}")
            try:
                urllib.request.urlretrieve(url, path)
            except Exception as e:
                print(f"ไม่สามารถดาวน์โหลด {url} ได้: {e}")

# ดาวน์โหลด corpus
print("กำลังดาวน์โหลดไฟล์...")
download_corpus()

# สร้าง hybrid search pipeline
print("กำลังตั้งค่า hybrid search pipeline...")
create_hybrid_search_pipeline()

# โหลดเอกสาร Markdown จากไดเรกทอรี
reader = SimpleDirectoryReader(
    input_dir="./corpus_input",
    recursive=True,
    required_exts=[".md", ".markdown"]
)
documents = reader.load_data()
print(f"โหลดเอกสาร {len(documents)} ไฟล์สำเร็จ")

# สร้าง parser สำหรับ Markdown
md_parser = MarkdownNodeParser()
nodes = md_parser.get_nodes_from_documents(documents)
print(f"สร้าง {len(nodes)} nodes ด้วย MarkdownNodeParser สำเร็จ")

# ตั้งค่า embedding model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3", device=device)
print(f"ตั้งค่าโมเดล embedding BAAI/bge-m3 สำเร็จ")

# ตรวจสอบขนาดของ embedding
embeddings = embed_model.get_text_embedding("test")
dim = len(embeddings)
print(f"ขนาด embedding: {dim}")

# ตั้งค่า OpensearchVectorClient
client = OpensearchVectorClient(
    endpoint=OPENSEARCH_ENDPOINT,
    index=OPENSEARCH_INDEX,
    dim=dim,
    embedding_field=EMBEDDING_FIELD,
    text_field=TEXT_FIELD,
    search_pipeline="hybrid-search-pipeline",
)
print(f"ตั้งค่า OpensearchVectorClient สำเร็จ สำหรับ index '{OPENSEARCH_INDEX}'")

# สร้าง vector store
vector_store = OpensearchVectorStore(client)

# สร้าง storage context
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# สร้าง index
index = VectorStoreIndex(
    nodes=nodes,
    storage_context=storage_context,
    embed_model=embed_model
)
print(f"สร้าง index สำเร็จ")

# บันทึก index ด้วย pickle
index_filename = f"{OPENSEARCH_INDEX}.pkl"
with open(index_filename, 'wb') as f:
    pickle.dump(index, f)
print(f"บันทึก index ลงในไฟล์ {index_filename} สำเร็จ")

print("เสร็จสิ้นกระบวนการทั้งหมด!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.7/261.7 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

ตั้งค่าโมเดล embedding BAAI/bge-m3 สำเร็จ


ขนาด embedding: 1024
ตั้งค่า OpensearchVectorClient สำเร็จ สำหรับ index 'aekanun_doc_index'
สร้าง index สำเร็จ


บันทึก index ลงในไฟล์ aekanun_doc_index.pkl สำเร็จ
เสร็จสิ้นกระบวนการทั้งหมด!
